![](../img/330-banner.png)

# Tutorial 5

UBC 2024-25

## Outline

During this tutorial, we will focus on calculating and understanding classification metrics, evaluating different classifiers, and addressing class imbalances.

All questions can be discussed with your classmates and the TAs - this is not a graded exercise!

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(".."), "code"))

import IPython
import matplotlib.pyplot as plt
import mglearn
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from plotting_functions import *
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

%matplotlib inline
pd.set_option("display.max_colwidth", 200)

from IPython.display import Image
pd.set_option("display.max_colwidth", 200)
DATA_DIR = "../data/"

For this exercise, we will use the same dataset on fraudulent and non-fraudulent transactions used in class (based on Kaggle's [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud) data set).

In [ ]:
# This dataset will be loaded using a URL instead of a CSV file
DATA_URL = "https://github.com/firasm/bits/raw/refs/heads/master/creditcard.csv"

cc_df = pd.read_csv(DATA_URL, encoding="latin-1")

Even our reduced version of this dataset is still large enough (almost 200k samples) that we can skip cross-validation, and create single training, validation and test sets:

In [ ]:
# First split to create the test set
train_df, test_df = train_test_split(cc_df, test_size=0.3, random_state=111)

X_train_big, y_train_big = train_df.drop(columns=["Class", "Time"]), train_df["Class"]

X_test, y_test = test_df.drop(columns=["Class", "Time"]), test_df["Class"] 

# Additional split to separate training and validation samples
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_big, y_train_big, test_size=0.3, random_state=123
)

We also know that this dataset is heavily unbalanced toward the negative class (non-fraud). Even a DummyClassifier would get more than 99% accuracy on this classification problem!

In [ ]:
train_df["Class"].value_counts(normalize=True)

### <font color='red'>Question 1</font>

Let's start by training a logistic regression model to separate the two classes included in this dataset. Then, observe the resulting confusion matrix, and compute **by hand** accuracy, recall and precision (you may use a calculator, but not the functions included in scikit-learn, like `classification_report`).

You may check your results using `classification_report` once you are done, but learning to calculate these metrics by looking at a confusion matrix is a good way to understand them.

Use the results to describe, for a non-expert audience, the behaviour of this classifier, answering questions such as:
- Is this classifier effective at finding fraudulent transactions?
- When this classifier reports a fraudulent transaction, are we confident that it is right?
- Is this classifier more likely to misclassify a fraud as a valid transaction, or vice-versa?

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay  

pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)
cm = ConfusionMatrixDisplay.from_estimator(
    pipe, X_valid, y_valid, values_format="d", display_labels=["Non fraud", "fraud"]
)

### <font color='red'>Question 2</font>

Next, we are going to try to address the problem of class imbalance as explained in this [video](https://www.youtube.com/watch?v=jHaKRCFb6Qw), by changing the weights of the samples.

Complete the code below to assign a weight of 50 to the positive class:

In [ ]:
pipe_lr_weight = make_pipeline(
    StandardScaler(), LogisticRegression(max_iter=500, class_weight={0: 1, 1: 50}) # add parameters here
)

pipe_lr_weight.fit(X_train, y_train)

### <font color='red'>Question 3</font>

Now, repeat the analysis you did in Question 1 (you can use `classification_report` this time). 

Again, imagine to be addressing a general audience: 
- How would you describe the performance of this classifier, compared to the first one we tried?
- Is this classifier better or worse at finding fraudulent transactions?
- When this classifier reports a fraudulent transaction, are we more or less confident that it is right?

### <font color='red'>Question 4</font>

Before settling on a model, we will make one more attempt at changing the weights of the classes, this time using the `class_weight="balanced"` parameter.

The code is provided below. The scikit-learn documentation says that:

- "If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y))". 

According to this explanation, what is now the relative weight of the positive class? 

In [ ]:
pipe_lr_balanced = make_pipeline(
    StandardScaler(), LogisticRegression(max_iter=500, class_weight="balanced")
)
pipe_lr_balanced.fit(X_train, y_train)

### <font color='red'>Question 5</font>

One more time, repeat your analysis, explaining your results to a general audience: 
- How would you describe the performance of this classifier, compared to the previous two?
- Is this classifier better or worse at finding fraudulent transactions?
- When this classifier reports a fraudulent transaction, are we more or less confident that it is right?

### <font color='red'>Question 6</font>

The lecture notes state:

- AUC can be interpreted as evaluating the **ranking** of positive examples.
- That is equivalent to the probability of a randomly picked positive point having a higher score according to the classifier than a randomly picked point from the negative class. 
- AUC of 1.0 means all positive points have a higher score than all negative points.

Based on these information, which one of the three classifiers that we trained do you expect to have the highest AUC score?

Compute the scores below to verify your answer!

## ML fairness activity

AI/ML systems can give the illusion of objectivity as they are derived from seemingly unbiased data & algorithm. However, human are inherently biased and AI/ML systems, if not carefully evaluated, can even further amplify the existing inequities and systemic bias in our society.  

How do we make sure our AI/ML systems are *fair*? Which metrics can we use to quantify 'fairness' in AI/ML systems?

Let's examine this on [the adult census data set](https://www.kaggle.com/uciml/adult-census-income). 

In [ ]:
census_df = pd.read_csv(DATA_DIR + "adult.csv")
census_df.shape

In [ ]:
train_df, test_df = train_test_split(census_df, test_size=0.4, random_state=42)

In [ ]:
train_df

### Preprocessing 

You may skip ahead or review as additional example of preprocessing.

In [ ]:
train_df_nan = train_df.replace("?", np.nan)
test_df_nan = test_df.replace("?", np.nan)
train_df_nan.shape

In [ ]:
# Let's identify numeric and categorical features

numeric_features = [
    "age",
    "capital.gain",
    "capital.loss",
    "hours.per.week",
]

categorical_features = [
    "workclass",
    "marital.status",
    "occupation",
    "relationship",
    "race",
    "native.country",
]

ordinal_features = ["education"]
binary_features = [
    "sex"
]  # Not binary in general but in this particular dataset it seems to have only two possible values
drop_features = ["education.num", "fnlwgt"]
target = "income"

In [ ]:
education_levels = [
    "Preschool",
    "1st-4th",
    "5th-6th",
    "7th-8th",
    "9th",
    "10th",
    "11th",
    "12th",
    "HS-grad",
    "Prof-school",
    "Assoc-voc",
    "Assoc-acdm",
    "Some-college",
    "Bachelors",
    "Masters",
    "Doctorate",
]

In [ ]:
assert set(education_levels) == set(train_df["education"].unique())

In [ ]:
X_train = train_df_nan.drop(columns=[target])
y_train = train_df_nan[target]

X_test = test_df_nan.drop(columns=[target])
y_test = test_df_nan[target]

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

numeric_transformer = make_pipeline(StandardScaler())

ordinal_transformer = OrdinalEncoder(categories=[education_levels], dtype=int)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore", sparse_output=False),
)

binary_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(drop="if_binary", dtype=int),
)

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (ordinal_transformer, ordinal_features),
    (binary_transformer, binary_features),
    (categorical_transformer, categorical_features),
    ("drop", drop_features),
)

### Model training and evaluation

This dataset is a bit unbalanced (see count below), so we are going to train a logistic regression model with `class_weight="balanced"` to account for this.

In [ ]:
y_train.value_counts()

In [ ]:
pipe_lr = make_pipeline(
    preprocessor, LogisticRegression(class_weight="balanced", max_iter=1000)
)

In [ ]:
pipe_lr.fit(X_train, y_train);

In [ ]:
ConfusionMatrixDisplay.from_estimator(pipe_lr, X_test, y_test);

Let's examine confusion matrix separately for the two genders we have in the data. 

In [ ]:
X_train_enc = preprocessor.fit_transform(X_train)
preprocessor.named_transformers_["pipeline-2"]["onehotencoder"].get_feature_names_out()

In [ ]:
X_female = X_test.query("sex=='Female'")  # X where sex is female
X_male = X_test.query("sex=='Male'")  # X where sex is male

y_female = y_test[X_female.index]  # y where sex is female
y_male = y_test[X_male.index]  # y where sex is male

**Get predictions for `X_female` and `y_male` with `pipe_lr`**

In [ ]:
female_preds = pipe_lr.predict(X_female)
male_preds = pipe_lr.predict(X_male)

Examine the accuracy and confusion matrix for female and male classes:

In [ ]:
print(classification_report(y_female, female_preds))

In [ ]:
print(classification_report(y_male, male_preds))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot the female confusion matrix
female_cm = ConfusionMatrixDisplay.from_estimator(pipe_lr, X_female, y_female, normalize="true");
axes[0].set_title('Confusion Matrix - Female');
female_cm.plot(ax=axes[0]);


# Plot the male confusion matrix
male_cm = ConfusionMatrixDisplay.from_estimator(pipe_lr, X_male, y_male, normalize="true");
axes[1].set_title('Confusion Matrix - Male');
male_cm.plot(ax=axes[1]);

### ❓❓ Questions for group discussion

Let's assume that a company is using this classifier for loan approval with a simple rule that if the income is >=50K, approve the loan else reject the loan. 

In your group, discuss the questions below. 

1. Which group has a higher accuracy?
2. Which group has a higher precision for class >50K? What about recall for class >50K?
3. Will both groups have more or less the same proportion of people with approved loans? 
4. If a male and a female have both a certain level of income, will they have the same chance of getting the loan?
5. Banks want to avoid approving unqualified applications (false positives) because default loan could have detrimental effects for them. Compare the false positive rates for the two groups.    
6. Overall, do you think this income classifier will fairly treat both groups? What will be the consequences of using this classifier in loan approval application? 
7. Do you think the effect will still exist if the sex feature is removed from the model (but you still have it available separately to do the two confusion matrices)? 
8. Are there any other groups in this dataset worth examining for biases? 
